In [1]:
import glob
import os

notes_path = "/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/*.csv"
glob.glob(os.path.expanduser(notes_path) + "*")

['/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/ICD9_Descriptions.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/MedCAT_Descriptions.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/SUBJECT_ID_to_ICD9.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/SUBJECT_ID_to_MedCAT.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/SUBJECT_ID_to_NAME.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/SUBJECT_ID_to_NOTES_1a.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/SUBJECT_ID_to_NOTES_1b.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/SUBJECT_ID_to_NOTES_templates.csv',
 '/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/reidentified_s

In [2]:
import pandas as pd

# discharge = pd.read_csv('/home/jxm3/mimic/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv')
df = pd.read_csv('/home/jxm3/mimic/physionet.org/files/clinical-bert-mimic-notes/1.0.0/setup_outputs/SUBJECT_ID_to_NOTES_1b.csv')

In [3]:
df.head()

,SUBJECT_ID,TEXT
0,2,Almeta Olivero nursing transfer note pt admitt...
1,2,Almeta Olivero neonatology attending triage no...
2,3,Kenton Lynn micu npn pm-pm: neuro: pt is restl...
3,3,Kenton Lynn micu nsg prog note: days remains s...
4,3,Kenton Lynn micu npn addendum: pt was agitated...


In [4]:
texts = df.sample(500)["TEXT"].tolist()
print(texts[0])

Reginia Kloster sicu hpi: m presented to osh on with fever/chills and bilateral hip pain l>r.
Reginia Kloster blood cx/left & right hip cx + staph. urine +mssa.
Reginia Kloster ct head negative at that time.
Reginia Kloster had left girtlestone, resection arthroplasty ()/right hip resection with insertion tobra cement spacer/girtlestone ().
Reginia Kloster post-op with large left cerebellar embolic stroke.
Reginia Kloster mssa hip infections and bacteremia.
Reginia Kloster pmhx: htn, dyslipidemia, djd, etoh + scotch/day current medications: acyclovir, acetaminophen, artificial tear ointment, calcium gluconate, chlorhexidine gluconate % oral rinse, dexmedetomidine, enoxaparin sodium, famotidine, fentanyl citrate, gentamicin, hydromorphone (dilaudid), hydralazine, insulin, labetalol, magnesium sulfate, midazolam, nafcillin, nystatin oral suspension, potassium chloride, propofol hour events: transthoracic echo - at : am fever - f - : pm allergies: tetracycline unknown; last dose of antibi

## Load model and evaluate

In [5]:
%load_ext autoreload
%autoreload 2

import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

In [6]:
import aliases

os.environ["TOKENIZERS_MULTIPROCESSING"] = "True"

# inv_trainer = aliases.load_trainer_from_alias("openai_msmarco__msl128__100epoch")
corr_experiment, corr_trainer = aliases.load_experiment_and_trainer_from_alias("gtr_nq__msl32_beta__correct")
inv_trainer = corr_trainer.inversion_trainer
# corr_trainer.precompute_hypotheses()
corr_trainer.model.eval()
print()

loading alias gtr_nq__msl32_beta__correct from /home/jxm3/research/retrieval/inversion/saves/47d9c149a8e827d0609abbeefdfd89ac...
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/47d9c149a8e827d0609abbeefdfd89ac/checkpoint-558000
set dataset to nq
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set num workers to 4
Overwriting max sequence length from 32 to 32
Overwriting use_less_data from None to -1
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Loading datasets with TOKENIZERS_PARALLELISM = False


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Renaming keys {'embedding_transform.2.bias', 'embedding_transform.2.weight'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 33])
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wy
================ End trainer sanity check ================
Froze 342572160 params from model type <class 'models.inversion.InversionModel'>


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Renaming keys {'embedding_transform.2.bias', 'embedding_transform.2.weight'} for backward compatibility.
================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 33])
	Decoded output -> The slithe and the tobogbes were mly; It wis grabbse tiring
================ End trainer sanity check ================



In [7]:
import datasets

temp_dataset = datasets.Dataset.from_dict({"text": texts})
datasets_dict = datasets.DatasetDict({
    "mimic": temp_dataset
})
d = corr_experiment._prepare_val_datasets_dict(
    model=corr_trainer.inversion_trainer.model,
    val_datasets_dict=datasets_dict,
    tokenizer=corr_trainer.tokenizer,
    embedder_tokenizer=corr_trainer.embedder_tokenizer,
)

Running tokenizer on dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
corr_trainer.num_gen_recursive_steps = 1
corr_trainer.evaluate(
    eval_dataset=d["mimic"],
    metric_key_prefix="mimic"
)

[pred] with bilateral ossia Regia Chili Edel / fever / hysteria with lodged hip pain on
[true] Reginia Kloster sicu hpi: m presented to osh on with fever/chills and bilateral hip pain



[pred] male of Rosalynn Yn Amo and is here with ephemeral chemotherapy. It is a hyl
[true] Rosalynn Eynon this is a yo male with hx of aml s/p chemotherapy, found to



[pred] Reichert: Aubery Reichert: Aubery unit name:, date of birth:, sex: no im
[true] Aubery Reichert name: reichert,aubery unit no: date of birth: sex: m service:


[pred] with bilateral ossia Regia Chili Edel / fever / hysteria with lodged hip pain on
[true] Reginia Kloster sicu hpi: m presented to osh on with fever/chills and bilateral hip pain



[pred] male of Rosalynn Yn Amo and is here with ephemeral chemotherapy. It is a hyl
[true] Rosalynn Eynon this is a yo male with hx of aml s/p chemotherapy, found to



[pred] Reichert: Aubery Reichert: Aubery unit name:, date of birth:, sex: no im
[true] Aubery Reichert name: reichert,aubery unit no: date of birth: sex: m service:


{'mimic_loss': 2.5207769870758057,
 'mimic_pred_num_tokens': 31.0,
 'mimic_true_num_tokens': 31.890625,
 'mimic_token_set_precision': 0.3816486500550947,
 'mimic_token_set_recall': 0.40827727173956785,
 'mimic_token_set_f1': 0.3922492689568483,
 'mimic_n_ngrams_match_1': 6.67,
 'mimic_n_ngrams_match_2': 1.624,
 'mimic_n_ngrams_match_3': 0.456,
 'mimic_num_true_words': 17.146,
 'mimic_num_pred_words': 16.602,
 'mimic_bleu_score': 6.046143082233343,
 'mimic_meteor_score': 0.30305510653678414,
 'mimic_rouge_score': 0.4018382236822939,
 'mimic_exact_match': 0.0,
 'mimic_emb_cos_sim': 0.8277257680892944,
 'mimic_runtime': 16.2315,
 'mimic_samples_per_second': 30.804,
 'mimic_steps_per_second': 0.123}

In [ ]:
corr_trainer.num_gen_recursive_steps = 50
corr_trainer.evaluate(
    eval_dataset=d["mimic"],
    metric_key_prefix="mimic"
)

generating from val:   0%|                                                                                                                                 | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
torch.cuda.empty_cache()

In [11]:
corr_trainer.args.per_device_eval_batch_size = 16
corr_trainer.return_best_hypothesis = True
corr_trainer.num_gen_recursive_steps = 50
corr_trainer.sequence_beam_width = 8

corr_trainer.evaluate(
    eval_dataset=d["mimic"].select(range(50)),
    metric_key_prefix="mimic",
)

step 48 // scores = [0.9449118971824646, 0.9613736867904663]


generating from val: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [13:50<00:00, 176.00s/it]

step 49 // scores = [0.9449118971824646, 0.9613736867904663]


[pred] Reginia Kloster sih osmi HW: was being presented with chills, fever, and bilateral hip pain on
[true] Reginia Kloster sicu hpi: m presented to osh on with fever/chills and bilateral hip pain



[pred] Rosalyn Eynon this is a yo male with hpn of aml x-s chemotherapy, found to
[true] Rosalynn Eynon this is a yo male with hx of aml s/p chemotherapy, found to



[pred] Aubery Reichert Aubery name: Reichert unit no.: service: sex: date of birth: m is
[true] Aubery Reichert name: reichert,aubery unit no: date of birth: sex: m service:


generating from val:   0%|                                                                                                                                 | 0/4 [00:00<?, ?it/s]

...

step 49 // scores = [0.9081634283065796, 1.0, 0.9762921333312988, 1.0, 0.9704880714416504, 0.9999998807907104, 0.9790281057357788, 0.9809747934341431, 1.0, 1.0, 0.9555952548980713, 0.9051269292831421, 0.9383915662765503, 0.9776409864425659, 0.9035502672195435, 0.9740313291549683]
step 0 // scores = [0.8158933520317078, 0.8601984977722168]
step 1 // scores = [0.9114115834236145, 0.9601646065711975]
step 2 // scores = [0.9165704250335693, 0.9632834196090698]
step 3 // scores = [0.9358375072479248, 0.9585325717926025]
step 4 // scores = [0.9300628900527954, 0.9616695046424866]
step 5 // scores = [0.9395490884780884, 0.9613736867904663]
step 6 // scores = [0.9348479509353638, 0.9613736867904663]
step 7 // scores = [0.9397881031036377, 0.9613736867904663]
step 8 // scores = [0.9415024518966675, 0.9613736867904663]
step 9 // scores = [0.9383856058120728, 0.9613736867904663]
step 10 // scores = [0.9387600421905518, 0.9613736867904663]
step 11 // scores = [0.9420363903045654, 0.961373686790466

generating from val: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [13:48<00:00, 175.92s/it]

step 49 // scores = [0.9449118971824646, 0.9613736867904663]


[pred] Reginia Kloster sih osmi HW: was being presented with chills, fever, and bilateral hip pain on
[true] Reginia Kloster sicu hpi: m presented to osh on with fever/chills and bilateral hip pain



[pred] Rosalyn Eynon this is a yo male with hpn of aml x-s chemotherapy, found to
[true] Rosalynn Eynon this is a yo male with hx of aml s/p chemotherapy, found to



[pred] Aubery Reichert Aubery name: Reichert unit no.: service: sex: date of birth: m is
[true] Aubery Reichert name: reichert,aubery unit no: date of birth: sex: m service:


{'mimic_loss': 2.675060749053955,
 'mimic_pred_num_tokens': 31.0,
 'mimic_true_num_tokens': 31.939998626708984,
 'mimic_token_set_precision': 0.7478289280379065,
 'mimic_token_set_recall': 0.7571698594401517,
 'mimic_token_set_f1': 0.7511563005942402,
 'mimic_n_ngrams_match_1': 12.94,
 'mimic_n_ngrams_match_2': 7.7,
 'mimic_n_ngrams_match_3': 5.34,
 'mimic_num_true_words': 17.24,
 'mimic_num_pred_words': 17.22,
 'mimic_bleu_score': 43.751149308120496,
 'mimic_meteor_score': 0.7064554221252809,
 'mimic_rouge_score': 0.796187672805147,
 'mimic_exact_match': 0.2,
 'mimic_emb_cos_sim': 0.967782735824585,
 'mimic_runtime': 1662.9228,
 'mimic_samples_per_second': 0.03,
 'mimic_steps_per_second': 0.002}

## measure name metrics


In [12]:
import torch

labels = torch.tensor(corr_trainer.preds_sample_labels_list)
preds = torch.tensor(corr_trainer.preds_sample_list)

# fix padding
eos_token_id = corr_trainer.embedder_tokenizer.eos_token_id
eos_tokens = (
    torch.ones(
        (len(preds), 1),
        dtype=torch.long,
        device=preds.device,
    )
    * eos_token_id
)
preds = torch.cat((preds[:, 1:], eos_tokens), dim=1)
assert labels.shape == preds.shape

In [13]:
from typing import Dict
import nltk


def evaluate_name_metrics(preds: torch.Tensor, labels: torch.Tensor) -> Dict[str, torch.Tensor]:
    pred_strs = corr_trainer.tokenizer.batch_decode(preds, skip_special_tokens=True)
    true_strs = corr_trainer.tokenizer.batch_decode(labels, skip_special_tokens=True)
    firsts_count = 0
    firsts_correct = 0
    lasts_count = 0
    lasts_correct = 0
    totals_count = 0
    totals_correct = 0
    for p, t in zip(pred_strs, true_strs):
        t = nltk.tokenize.word_tokenize(t)
        first, last = t[0], t[1]
        firsts_count += t.count(first)
        lasts_count += t.count(last)
        totals_count += "__".join(t).count("__".join([first, last]))
        #     
        p = nltk.tokenize.word_tokenize(p)
        firsts_correct += p.count(first)
        lasts_correct += p.count(last)
        totals_correct += "__".join(p).count("__".join([first, last]))
    return { "first": (firsts_correct / firsts_count), "last": (lasts_correct / lasts_count), "full": (totals_correct / totals_count) }

evaluate_name_metrics(preds, labels)

{'first': 0.9166666666666666,
 'last': 0.8421052631578947,
 'full': 0.7543859649122807}

In [15]:
corr_trainer.inversion_trainer.gen_kwargs

{'early_stopping': False,
 'num_beams': 1,
 'do_sample': False,
 'no_repeat_ngram_size': 0}

In [16]:
## now do single-step

corr_trainer.args.per_device_eval_batch_size = 16
corr_trainer.return_best_hypothesis = True
corr_trainer.num_gen_recursive_steps = 50
corr_trainer.sequence_beam_width = 8

corr_trainer.inversion_trainer.evaluate(
    eval_dataset=d["mimic"].select(range(500)),
    metric_key_prefix="mimic",
)

[pred] with bilateral hip ossia and rheumatoid arthritis Regia Klu - fMRI pain at home with
[true] Reginia Kloster sicu hpi: m presented to osh on with fever/chills and bilateral hip pain



[pred] a male of Rosalyn Yn amo (Hyl esote). He has XML editing and has
[true] Rosalynn Eynon this is a yo male with hx of aml s/p chemotherapy, found to



[pred] Reichert name: Aubery Reichert sex, date of birth, unit of service, sex and sex of
[true] Aubery Reichert name: reichert,aubery unit no: date of birth: sex: m service:


{'mimic_loss': 2.7380549907684326,
 'mimic_accuracy': 0.4318033854166667,
 'mimic_pred_num_tokens': 31.0,
 'mimic_true_num_tokens': 31.890625,
 'mimic_token_set_precision': 0.30447188321987734,
 'mimic_token_set_recall': 0.3348138985840214,
 'mimic_token_set_f1': 0.3165762396403704,
 'mimic_n_ngrams_match_1': 5.338541666666667,
 'mimic_n_ngrams_match_2': 1.0859375,
 'mimic_n_ngrams_match_3': 0.28125,
 'mimic_num_true_words': 17.1328125,
 'mimic_num_pred_words': 16.4296875,
 'mimic_bleu_score': 3.916002736499548,
 'mimic_meteor_score': 0.23425502943117402,
 'mimic_rouge_score': 0.31694972995276594,
 'mimic_exact_match': 0.0,
 'mimic_emb_cos_sim': 0.7712833881378174,
 'mimic_perplexity': 15.456892042479177,
 'mimic_runtime': 5.503,
 'mimic_samples_per_second': 90.859,
 'mimic_steps_per_second': 0.727}

In [17]:
import torch

labels = torch.tensor(inv_trainer.preds_sample_labels_list)
preds = torch.tensor(inv_trainer.preds_sample_list)

# fix padding
eos_token_id = corr_trainer.embedder_tokenizer.eos_token_id
eos_tokens = (
    torch.ones(
        (len(preds), 1),
        dtype=torch.long,
        device=preds.device,
    )
    * eos_token_id
)
preds = torch.cat((preds[:, 1:], eos_tokens), dim=1)
assert labels.shape == preds.shape

evaluate_name_metrics(preds, labels)

{'first': 0.4, 'last': 0.2807377049180328, 'full': 0.125}

## Trying a longer experiment

In [ ]:
corr_trainer.args.per_device_eval_batch_size = 8
corr_trainer.return_best_hypothesis = True
corr_trainer.num_gen_recursive_steps = 500
corr_trainer.sequence_beam_width = 8

corr_trainer.evaluate(
    eval_dataset=d["mimic"].select(range(100)),
    metric_key_prefix="mimic",
)

In [ ]:
corr_trainer